<a href="https://colab.research.google.com/github/AlkaidCheng/quple.github.io/blob/master/examples/Variational_Circuits_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install quple

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.9MB 42.1MB/s 
     |████████████████████████████████| 1.4MB 30.1MB/s 
     |████████████████████████████████| 3.9MB 40.6MB/s 
     |████████████████████████████████| 450kB 46.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 163kB 50.6MB/s 
     |████████████████████████████████| 5.3MB 41.4MB/s 
     |████████████████████████████████| 112kB 50.3MB/s 
     |████████████████████████████████| 133kB 43.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=868b8c08c8858c80e19bd4c603d008bbb77d446debc59233abdcf274c9438f23
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for pathos: filename=pathos-0.2.5-cp36-none-any.whl size=77577 sha256=fb1c2aef0b5c936043b416c45e3d81662fe545bec2bfc8149

# Variational Circuit

A variational (or model) circuit is a parameterized quantum circuit representing a unitary $U(\theta)$ parameterized by a set of free parameters $\theta$ which are treated as the weights in a machine learing model.

 The variational circuit implemented by Quple is based on a circuit-centric design. In this architecture, a variational circuit of $n$ qubits is composed of $L$ copies (i.e. the circuit depth) of a primary circuit block. Each circuit block consists of a layer of single qubit gates (the rotation layer) applied to each of the qubits, followed by a layer of two qubit gates (the entanglement layer) to entangle all qubits according to a given interaction graph. A final rotation layer is added to the circuit so that measurement on any of the data qubits will effectively include the effect of all the two qubit gates in the entanglement layer. It is an example of a strongly entangling circuit which has the advantage of capturing correlations between the data features at all ranges for a short range circuit. 
	
The class that implements the variational circuits are
* RealAmplitudes
* EfficientSU2
* ExcitationPreserving
* IsingCoupling

In [2]:
from quple.circuits.variational_circuits import RealAmplitudes, EfficientSU2, ExcitationPreserving, IsingCoupling

### Variational Circuit - RealAmplitudes
The `RealAmplitudes` circuit consists of a layer of single qubit Pauli Y rotations acting on each qubit followed by a layer of CNOT entanglement on pairs of qubits under a given interaction graph. The resultant quantum state from the RealAmplitudes circuit will only have real amplitudes with zero complex part. It is a hardware efficient circuit as it uses entangling interactions that are naturally available on hardware and do not require compilation. 

Arguments:
* n_qubit (int): Number of qubits in the circuit
* copies (int): Number of times the circuit layers are repeated.
* entangle_strategy (default='full'): Determines how the qubits are connected in an entanglement block.
* parameter_symbol (str): Symbol prefix for circuit parameters.

In [3]:
# RealAmplitudes variational circuit with 5 qubits and circuit depth 2 constructed from RY rotations and fully connected CNOT entanglement 
variational_circuit = RealAmplitudes(n_qubit=5, copies=2)
print(variational_circuit)

                                 ┌──┐       ┌──┐                                   ┌──┐       ┌──┐
(0, 0): ───Ry(θ_0)───@───@───@────@────────────────────Ry(θ_5)───@─────────@───@────@────────────────────Ry(θ_10)──────────────
                     │   │   │    │                              │         │   │    │
(0, 1): ───Ry(θ_1)───X───┼───┼────┼@────@────@─────────Ry(θ_6)───X─────────┼───┼────┼@────@────@─────────Ry(θ_11)──────────────
                         │   │    ││    │    │                             │   │    ││    │    │
(0, 2): ───Ry(θ_2)───────X───┼────┼X────┼────┼@────@───Ry(θ_7)─────────────X───┼────┼X────┼────┼@────@───Ry(θ_12)──────────────
                             │    │     │    ││    │                           │    │     │    ││    │
(0, 3): ───Ry(θ_3)───────────X────┼─────X────┼X────┼───@─────────Ry(θ_8)───────X────┼─────X────┼X────┼───@──────────Ry(θ_13)───
                                  │          │     │   │                            │          │     │ 

### Variational Circuit - EfficientSU2
The `EfficientSU2` circuit consists of a layer of single qubit operations spanned by SU(2) (such as the Pauli X, Y and Y operations and their rotations) acting on each qubit and a layer of CNOT entanglement on pairs of qubits under a given interaction graph. It is a hardware efficient circuit as it uses entangling interactions that are naturally available on hardware and do not require compilation. 

Arguments:
* n_qubit (int): Number of qubits in the circuit
* su2_gates (default=['RY','RZ']): The SU(2) gates to be used in the circuit.
* copies (int): Number of times the circuit layers are repeated.
* entangle_strategy (default='full'): Determines how the qubits are connected in an entanglement block.
* parameter_symbol (str): Symbol prefix for circuit parameters.

In [4]:
# EfficientSU2 variational circuit with 5 qubits and circuit depth 2 constructed from ['RY', 'RZ'] rotations and linearly connected CNOT entanglement repeated 2 times
variational_circuit = EfficientSU2(n_qubit=5, copies=2, entangle_strategy='linear')
print(variational_circuit)

(0, 0): ───Ry(θ_0)───Rz(θ_5)───@───────────Ry(θ_10)───Rz(θ_15)───@──────────────────Ry(θ_20)───Rz(θ_25)──────────────
                               │                                 │
(0, 1): ───Ry(θ_1)───Rz(θ_6)───X───@───────Ry(θ_11)───Rz(θ_16)───X──────────@───────Ry(θ_21)───Rz(θ_26)──────────────
                                   │                                        │
(0, 2): ───Ry(θ_2)───Rz(θ_7)───────X───@───Ry(θ_12)───Rz(θ_17)──────────────X───@───Ry(θ_22)───Rz(θ_27)──────────────
                                       │                                        │
(0, 3): ───Ry(θ_3)───Rz(θ_8)───────────X───@──────────Ry(θ_13)───Rz(θ_18)───────X───@──────────Ry(θ_23)───Rz(θ_28)───
                                           │                                        │
(0, 4): ───Ry(θ_4)───Rz(θ_9)───────────────X──────────Ry(θ_14)───Rz(θ_19)───────────X──────────Ry(θ_24)───Rz(θ_29)───


### Variational Circuit - ExcitationPreserving

The `ExcitationPreserving` circuit consists of a layer of single qubit Pauli Z rotation acting on each qubit  and a layer of two qubit gates from the Fermionic simulation, or fSim, gate set acting on pairs of qubits under a given interaction graph. Under this gate set, the $\sigma_X\sigma_X$ and $\sigma_Y\sigma_Y$ couplings between the qubits have equal coefficients which conserves the number of excitations of the qubits.  Algorithms performed with just Pauli Z rotations and fSim gates enable error mitigation techiques including post selection and zero noise extrapolation.

Arguments:
* n_qubit (int): Number of qubits in the circuit
* entanglement_gate ('RISWAP' or 'FSim', default='RISWAP'): Excitation preserving gate operation to use for entangling qubits
* copies (int): Number of times the circuit layers are repeated.
* entangle_strategy (default='full'): Determines how the qubits are connected in an entanglement block.
* parameter_symbol (str): Symbol prefix for circuit parameters.

In [5]:
# ExcitationPreserving variational circuit with 5 qubits and circuit depth 1 constructed from RZ rotation and circularly connected RISWAP entanglement
variational_circuit = ExcitationPreserving(n_qubit=5, copies=1, entangle_strategy='circular', parameter_symbol='ɸ')
print(variational_circuit)

(0, 0): ───Rz(ɸ_0)───iSwap───────────────────────────────────────────────────────────────────────iSwap──────────────Rz(ɸ_10)───
                     │                                                                           │
(0, 1): ───Rz(ɸ_1)───iSwap^(2*ɸ_5/pi)───iSwap────────────────────────────────────────────────────┼──────────────────Rz(ɸ_11)───
                                        │                                                        │
(0, 2): ───Rz(ɸ_2)──────────────────────iSwap^(2*ɸ_6/pi)───iSwap─────────────────────────────────┼──────────────────Rz(ɸ_12)───
                                                           │                                     │
(0, 3): ───Rz(ɸ_3)─────────────────────────────────────────iSwap^(2*ɸ_7/pi)───iSwap──────────────┼──────────────────Rz(ɸ_13)───
                                                                              │                  │
(0, 4): ───Rz(ɸ_4)────────────────────────────────────────────────────────────iSwap^(2*ɸ_8/p

In [6]:
# ExcitationPreserving variational circuit with 5 qubits and circuit depth 1 constructed from RZ rotation and star connected FSim entanglement
variational_circuit = ExcitationPreserving(n_qubit=5, copies=1, entanglement_gate='FSim', entangle_strategy='star')
print(variational_circuit)

(0, 0): ───Rz(θ_0)───fsim(θ_5, θ_6)───fsim(θ_7, θ_8)───fsim(θ_9, θ_10)───fsim(θ_11, θ_12)───Rz(θ_13)───
                     │                │                │                 │
(0, 1): ───Rz(θ_1)───#2───────────────┼────────────────┼─────────────────┼──────────────────Rz(θ_14)───
                                      │                │                 │
(0, 2): ───Rz(θ_2)────────────────────#2───────────────┼─────────────────┼──────────────────Rz(θ_15)───
                                                       │                 │
(0, 3): ───Rz(θ_3)─────────────────────────────────────#2────────────────┼──────────────────Rz(θ_16)───
                                                                         │
(0, 4): ───Rz(θ_4)───────────────────────────────────────────────────────#2─────────────────Rz(θ_17)───


### Variational Circuit - IsingCoupling

The `IsingCoupling` circuit consists of a layer of single qubit Pauli rotations acting on each qubit and a layer of two qubit XX Ising coupling gates which is a rotation around the $XX$ axis in the two-qubit bloch sphere. 

Arguments:

* n_qubit (int): Number of qubits in the circuit
* rotation_gates (default=['RY','RZ']): Rotation gates to be used in the circuit.
* copies (int): Number of times the circuit layers are repeated.
* entangle_strategy (default='full'): Determines how the qubits are connected in an entanglement block.
* parameter_symbol (str): Symbol prefix for circuit parameters.

In [7]:
# IsingCoupling variational circuit with 5 qubits and circuit depth 1 constructed from ['RY','RZ'] rotation and fully connected XX entanglement raised to some parameterised power
variational_circuit = IsingCoupling(n_qubit=5, copies=1, entangle_strategy='full')
print(variational_circuit)

                                                                   ┌──────────────────┐               ┌──────────────────┐
(0, 0): ───Ry(θ_0)───Rz(θ_5)───XX──────────XX──────────XX───────────XX───────────────────────────────────────────────────────────────────Ry(θ_20)────Rz(θ_25)──────────────
                               │           │           │            │
(0, 1): ───Ry(θ_1)───Rz(θ_6)───XX^(θ_10)───┼───────────┼────────────┼────────XX───────────XX───────────XX────────────────────────────────Ry(θ_21)────Rz(θ_26)──────────────
                                           │           │            │        │            │            │
(0, 2): ───Ry(θ_2)───Rz(θ_7)───────────────XX^(θ_11)───┼────────────┼────────XX^(θ_14)────┼────────────┼────────XX───────────XX──────────Ry(θ_22)────Rz(θ_27)──────────────
                                                       │            │                     │            │        │            │
(0, 3): ───Ry(θ_3)───Rz(θ_8)───────────────────────────XX^(

## Readout qubit

Addition of an extra readout qubit that is entangled to all data qubits via a custom two-qubit gate operation which is used as the qubit for measurement when training a VQC model. 

In [8]:
# EfficientSU2 variational circuit with 5 qubits and circuit depth 2 constructed from ['RY', 'RZ'] rotations and linearly connected CNOT entanglement repeated 2 times
variational_circuit = EfficientSU2(n_qubit=5, copies=2, entangle_strategy='linear')
print(variational_circuit)
print('\n')
# Add a readout qubit with XX entanglement (raised to some parameterised power, i.e. roughly equal to RXX)
variational_circuit.add_readout('XX')
print(variational_circuit)

(0, 0): ───Ry(θ_0)───Rz(θ_5)───@───────────Ry(θ_10)───Rz(θ_15)───@──────────────────Ry(θ_20)───Rz(θ_25)──────────────
                               │                                 │
(0, 1): ───Ry(θ_1)───Rz(θ_6)───X───@───────Ry(θ_11)───Rz(θ_16)───X──────────@───────Ry(θ_21)───Rz(θ_26)──────────────
                                   │                                        │
(0, 2): ───Ry(θ_2)───Rz(θ_7)───────X───@───Ry(θ_12)───Rz(θ_17)──────────────X───@───Ry(θ_22)───Rz(θ_27)──────────────
                                       │                                        │
(0, 3): ───Ry(θ_3)───Rz(θ_8)───────────X───@──────────Ry(θ_13)───Rz(θ_18)───────X───@──────────Ry(θ_23)───Rz(θ_28)───
                                           │                                        │
(0, 4): ───Ry(θ_4)───Rz(θ_9)───────────────X──────────Ry(θ_14)───Rz(θ_19)───────────X──────────Ry(θ_24)───Rz(θ_29)───


(-1, -1): ─────────────────────────────────────────────────────────────────────────────────────